<a href="https://colab.research.google.com/github/saleh1312/nlp_projects/blob/main/Seq2Seq%20image%20translation%20%2B%20Attintion%20layer%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy
import string
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy
import tensorflow as tf
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [2]:
with open(r"/content/drive/MyDrive/data/small_vocab_en.txt", "r") as f:
    en_sent=f.read().split('\n')

with open(r"/content/drive/MyDrive/data/small_vocab_fr.txt", "r") as f:
    fr_sent=f.read().split('\n')


In [3]:
en_sent_clean=[]
for sent in en_sent:
    word_tok=word_tokenize(sent,'english')
    new_word=[]
    for token in word_tok:
        if token not in string.punctuation:
            new_word.append(token)
    en_sent_clean.append((' '.join(new_word)).lower()+' eos')

In [4]:
en_sent_clean[:3]

['new jersey is sometimes quiet during autumn and it is snowy in april eos',
 'the united states is usually chilly during july and it is usually freezing in november eos',
 'california is usually quiet during march and it is usually hot in june eos']

In [5]:
fr_sent_clean=[]
for sent in fr_sent:
    word_tok=word_tokenize(sent,'french')
    new_word=[]
    for token in word_tok:
        if token not in string.punctuation:
            new_word.append(token)
    fr_sent_clean.append('sos '+((' '.join(new_word)).lower()+' eos'))

In [6]:
fr_sent_clean[:3]

['sos new jersey est parfois calme pendant l automne et il est neigeux en avril eos',
 'sos les états-unis est généralement froid en juillet et il gèle habituellement en novembre eos',
 'sos california est généralement calme en mars et il est généralement chaud en juin eos']

In [7]:
del en_sent
del fr_sent

In [8]:
toknizer_en  = tf.keras.preprocessing.text.Tokenizer()
toknizer_en.fit_on_texts(en_sent_clean)
sequences_en = toknizer_en.texts_to_sequences(en_sent_clean)

In [9]:
toknizer_fr  = tf.keras.preprocessing.text.Tokenizer()
toknizer_fr.fit_on_texts(fr_sent_clean)
sequences_fr = toknizer_fr.texts_to_sequences(fr_sent_clean)

In [10]:
sequences_en[:3]

[[18, 24, 1, 9, 68, 5, 40, 8, 4, 1, 56, 3, 45, 2],
 [6, 21, 22, 1, 10, 63, 5, 44, 8, 4, 1, 10, 52, 3, 46, 2],
 [23, 1, 10, 68, 5, 39, 8, 4, 1, 10, 69, 3, 35, 2]]

In [11]:
sequences_fr[:3]

[[2, 37, 36, 1, 10, 69, 39, 13, 26, 8, 5, 1, 114, 4, 52, 3],
 [2, 6, 34, 33, 1, 14, 21, 4, 51, 8, 5, 97, 71, 4, 53, 3],
 [2, 103, 1, 14, 69, 4, 47, 8, 5, 1, 14, 23, 4, 43, 3]]

In [12]:
frinput=[]
froutput=[]
for seq in sequences_fr:
    seqx=seq[:-1]
    seqy=seq[1:]
    frinput.append(seqx)
    froutput.append(seqy)

In [13]:
t1= max([len(w) for w in sequences_en])
t2= max([len(w) for w in sequences_fr])
maxlen=max([t1,t2])

In [14]:
pad_en=tf.keras.preprocessing.sequence.pad_sequences(sequences_en, padding='post',maxlen=maxlen)
pad_frinput=tf.keras.preprocessing.sequence.pad_sequences(frinput, padding='post',maxlen=maxlen)
pad_froutput=tf.keras.preprocessing.sequence.pad_sequences(froutput, padding='post',maxlen=maxlen)

In [15]:
hot_pad_fr=tf.keras.utils.to_categorical(pad_froutput)

In [16]:
from sklearn.model_selection import train_test_split
#[pad_en,pad_frinput],hot_pad_fr
xt1,xs1,xt2,xs2,yt1,ys1=train_test_split(pad_en,pad_frinput,hot_pad_fr, test_size=0.1)

In [17]:
del hot_pad_fr
del pad_frinput
del pad_en

In [30]:
import gc
gc.collect()

18064

In [19]:
input= tf.keras.Input((maxlen))
embd=tf.keras.layers.Embedding(len(toknizer_en.word_index)+1,7,input_length=maxlen)(input)
h0,h1,c=tf.keras.layers.LSTM(100,return_sequences=True,return_state=True)(embd)



input2= tf.keras.Input((maxlen))
decoder_embedding=tf.keras.layers.Embedding(len(toknizer_fr.word_index)+1,7,input_length=maxlen)(input2)

decoder_out=tf.keras.layers.LSTM(100,return_sequences=True)(decoder_embedding, initial_state=[h1,c])

attn_out, attn_states = tf.keras.layers.Attention()(
    [decoder_out, h0],return_attention_scores=True)

soft=tf.keras.layers.Softmax(axis=-1)(attn_states)



decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_out, attn_out])

d=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(toknizer_fr.word_index)+1
                                                        ,activation='softmax'))(decoder_concat_input)
model=tf.keras.Model([input,input2],d)
model2=tf.keras.Model([input,input2],[d,soft])
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 23, 7)        1393        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 23, 7)        2429        ['input_2[0][0]']                
                                                                                            

In [20]:

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=
              tf.keras.optimizers.Adam(0.001))

model2.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=
              tf.keras.optimizers.Adam(0.001))

In [ ]:
xt1,xs1,xt2,xs2,yt1,ys1
model.fit([xt1,xt2],yt1,epochs=100,batch_size=32,validation_data=
          ([xs1,xs2],ys1))

In [32]:
model2.save('/content/drive/MyDrive/modeldd.h5')

In [36]:


r=np.random.randint(0,len(en_sent_clean)-1)
entext=en_sent_clean[r]
ground=fr_sent_clean[r]
print(entext)
print(ground)


entoken=toknizer_en.texts_to_sequences([entext])
entokenpad=tf.keras.preprocessing.sequence.pad_sequences(entoken, padding='post',maxlen=maxlen)

frtext='sos'
i=0
attnsion=[]
while True:
  frtoken=toknizer_fr.texts_to_sequences([frtext])
  frtokenpad=tf.keras.preprocessing.sequence.pad_sequences(frtoken, padding='post',maxlen=maxlen)
  preds,attn=model2.predict([entokenpad,frtokenpad])
  word_attn=attn[0,i]
  index=np.argmax(preds[0,i])
  word=list(toknizer_fr.word_index.keys())[list(toknizer_fr.word_index.values()).index(index)]
  if word == 'eos':
    frtext=frtext+' '+'eos'
    attnsion.append(word_attn.tolist())
    break
  frtext=frtext+' '+word
  attnsion.append(word_attn.tolist())
  i+=1
print(frtext)
att_n=np.array(attnsion)
entextpad= entext.split()

while True:
  if len( entextpad ) != maxlen:
    entextpad.append(' <pad>')
  else:
    break
import plotly.graph_objects as go
xmin=10
fig = go.Figure(data=go.Heatmap(
                    z=att_n,
                    x=entextpad,
                    y=frtext.split()[1:]))
fig['layout'].update(plot_bgcolor='black')
fig.show()

the united states is busy during january and it is never relaxing in spring eos
sos les états-unis est occupé en janvier et il est jamais relaxant au printemps eos
sos les états unis est occupé en janvier et il est jamais relaxant au printemps eos
